In [1]:
cd /home/sumin/Downloads/jit

/home/sumin/Downloads/jit


In [2]:
from pickle import FALSE
#-*- coding: utf-8 -*-
'''
Train bpe model and bpe-segment train/valid/test datasets.
e.g.,
python bpe_segment.py --jit /content/drive/MyDrive/Colab Notebooks/jit --vocab_size 4000
'''
import codecs
import os
import sentencepiece as spm

from jamo import h2j # syllable to jamo (letter)
import argparse
import easydict

def train_bpe(fpath, vocab_size):
    dir = os.path.dirname(fpath)
    train = f'--input={fpath} \
              --normalization_rule_name=identity \
              --model_prefix={dir}/bpe \
              --character_coverage=1 \
              --vocab_size={vocab_size} \
              --model_type=bpe'
    spm.SentencePieceTrainer.Train(train)

    # modify Dictionary
    lines = [line.replace("\t", " ") for line in codecs.open(f'{dir}/bpe.vocab', 'r', 'utf8').read().splitlines()[3:]]
    with codecs.open(f'{dir}/bpe.dict', 'w', 'utf8') as fout:
        fout.write("\n".join(lines))
    os.system(f'rm {dir}/bpe.vocab')

def apply_bpe(sp, sents, out_file):
    with codecs.open(out_file, 'w', 'utf8') as fout:
        fout.write("\n".join(" ".join(sp.EncodeAsPieces(sent)) for sent in sents))


if __name__ == "__main__":
    # arguments setting
    hp = easydict.EasyDict({
        "jit" : "/home/sumin/Downloads/jit",
        "vocab_size" : 4000
    })

In [3]:
# train/valid/test
sample = codecs.open(f"{hp.jit}/sample.txt", 'r', 'utf8').read().splitlines()

    # bpe train
dir = 'data/{}k/bpe'.format(str(hp.vocab_size)[:-3])
os.makedirs(dir, exist_ok=True)

    # apply
sp = spm.SentencePieceProcessor()
sp.Load(f'{dir}/bpe.model')
apply_bpe(sp, sample, f'{dir}/sample.txt')

!fairseq-interactive /home/sumin/Downloads/jit/data/4k/ko-je-bin \
  --path /home/sumin/train/4k/ko-je-bin/ckpt/checkpoint_best.pt \
  --source-lang ko --target-lang je \
  --remove-bpe 'sentencepiece' \
  --beam 5 \
  --input={dir}/sample.txt > target.txt 

2023-06-17 16:03:37 | INFO | fairseq_cli.interactive | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False, 'reset_logging': False, 'suppress_crashes': False, 'use_plasma_view': False, 'plasma_path': '/tmp/plasma'}, 'common_eval': {'_na

In [9]:
import torch